In [1]:
import pandas as pd
import numpy as np
import glob
import os

Data Source https://github.com/buds-lab/building-data-genome-project-2


In [2]:
data_dir = r"/Users/hamnaj/Documents/GitHub/DECODER-Project/data"

# Read all .txt files in the folder
files = glob.glob(os.path.join(data_dir, "*.csv"))

In [3]:
all_data = []

for file in files:
    meter_type = os.path.basename(file).replace(".csv", "")
    print(f"Reading {meter_type} ...")

    # Read as comma-separated
    df = pd.read_csv(file, sep=',')
    df['meter_type'] = meter_type
    all_data.append(df)

Reading hotwater ...
Reading gas ...
Reading solar ...
Reading electricity ...
Reading water ...
Reading chilledwater ...
Reading irrigation ...
Reading steam ...


In [4]:
# Combine all into one DataFrame
data = pd.concat(all_data, ignore_index=True)

print("✅ Combined shape:", data.shape)
print(data.head())

# Save combined data for reference
output_path = os.path.join(data_dir, "combined_data_preview.csv")
data.to_csv(output_path, index=False)
print(f"✅ Combined file saved at: {output_path}")

✅ Combined shape: (140352, 1638)
             timestamp  Robin_public_Carolina  Robin_office_Wai  \
0  2016-01-01 00:00:00                 0.2421               0.0   
1  2016-01-01 01:00:00                 9.7579             100.0   
2  2016-01-01 02:00:00                 0.0000             100.0   
3  2016-01-01 03:00:00                 0.0000               0.0   
4  2016-01-01 04:00:00                10.0000             100.0   

   Robin_office_Zelma  Robin_office_Lindsay  Robin_education_Derick  \
0              0.0000                9.7989                299.7292   
1              0.0000               10.0000                593.0020   
2              0.0000               10.0000                600.0000   
3            100.0000               10.0000                600.0000   
4            201.8749               10.0000                600.0000   

   Robin_education_Cecilia  Robin_education_Della  Robin_education_Leslie  \
0                  19.6809                    0.0           

In [5]:
# Keep only rows that have a timestamp column
if 'timestamp' not in data.columns:
    raise ValueError("No 'timestamp' column found — check your input files.")

# Melt (unpivot) so that each reading is one row
data_long = data.melt(
    id_vars=['timestamp', 'meter_type'],
    var_name='building_id',
    value_name='value'
)

print("✅ Long-format shape:", data_long.shape)
print(data_long.head())

# Save a smaller sample for testing the API
sample = data_long.sample(5000, random_state=42)
sample.to_csv("sample_sensor_data.csv", index=False)
print("📁 Saved sample_sensor_data.csv (for API testing)")


✅ Long-format shape: (229615872, 4)
             timestamp meter_type            building_id    value
0  2016-01-01 00:00:00   hotwater  Robin_public_Carolina   0.2421
1  2016-01-01 01:00:00   hotwater  Robin_public_Carolina   9.7579
2  2016-01-01 02:00:00   hotwater  Robin_public_Carolina   0.0000
3  2016-01-01 03:00:00   hotwater  Robin_public_Carolina   0.0000
4  2016-01-01 04:00:00   hotwater  Robin_public_Carolina  10.0000
📁 Saved sample_sensor_data.csv (for API testing)


In [6]:
data_long.info()
data_long.describe()
data_long['meter_type'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229615872 entries, 0 to 229615871
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   timestamp    object 
 1   meter_type   object 
 2   building_id  object 
 3   value        float64
dtypes: float64(1), object(3)
memory usage: 6.8+ GB


meter_type
hotwater        28701984
gas             28701984
solar           28701984
electricity     28701984
water           28701984
chilledwater    28701984
irrigation      28701984
steam           28701984
Name: count, dtype: int64

In [2]:
import sys, os
import pandas as pd

sys.path.append(os.path.abspath(".."))
from app.database import Base, engine
from app.models import SensorData


Base.metadata.create_all(bind=engine)


In [4]:
import pandas as pd
from app.database import SessionLocal
from app.models import SensorData
from datetime import datetime


data["timestamp"] = pd.to_datetime(data["timestamp"], errors="coerce", utc=True)
data["value"] = pd.to_numeric(data["value"], errors="coerce")
data = data.dropna(subset=["timestamp", "value"])

session = SessionLocal()
for _, row in data.iterrows():
    entry = SensorData(
        building_id=row["building_id"],
        meter_type=row["meter_type"],
        timestamp=row["timestamp"].to_pydatetime(),
        value=row["value"]
    )
    session.add(entry)
session.commit()
session.close()
